In [ ]:
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, ResNet101, MobileNet, EfficientNetB7
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from google.colab import drive
import tensorflow as tf



drive.mount('/content/drive')

In [ ]:
tf.test.gpu_device_name()
!/opt/bin/nvidia-smi

In [ ]:
!unrar x '/content/drive/MyDrive/Base de datos/human_data.rar' "/content/sample_data/"

#!unzip '/content/drive/MyDrive/Base de datos/aumento_tumor.zip' -d "/content/sample_data/"

In [ ]:
def generador_datos(input_dir, target_dir, img_s, b_size= 16, tipo_c= 'categorical', img_c= 'rgb'):

  generador_ent= ImageDataGenerator(rescale= 1/255.0)
  generador_val= ImageDataGenerator(rescale= 1/255.0)


  ent_gen= generador_ent.flow_from_directory(input_dir,
                                            class_mode= tipo_c,
                                            target_size= img_s,
                                            batch_size= b_size,
                                            color_mode= img_c,
                                            shuffle= True,
                                            )

  val_gen= generador_val.flow_from_directory(target_dir,
                                            class_mode= tipo_c,
                                            target_size= img_s,
                                            color_mode= img_c,
                                            batch_size= b_size,
                                            shuffle= True,
                                            )
  
  return ent_gen, val_gen

In [ ]:
def func(name_model):
    print('Modelo seleccionado => {} '.format(name_model))
    
    pre_model = name_model(input_shape=(200, 200, 3),
                   include_top= False,
                   weights= 'imagenet',
                   pooling= 'avg')
    
    pre_model.trainable = True
    inputs = pre_model.input
    x = Dense(1024, activation='relu')(pre_model.output)
    x = Dense(1024, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
def ent_c(model, epocas, ent_gen, val_gen, opt= 'Adam', perd= 'categorical_crossentropy', resumen= False):
  
  #checkpoint= ModelCheckpoint("VGG19.hdf5", monitor= 'val_loss', save_best_only= True, mode= 'min', save_weights_only= False)
  model.compile(loss = perd, optimizer= opt, metrics=['accuracy'])
  model.summary() if resumen== True else None
  history = model.fit(ent_gen, validation_data=val_gen, epochs=epocas)#, callbacks= checkpoint)

In [ ]:
ent_gen, val_gen= generador_datos('/content/sample_data/content/sample_data/aumento/ent',
                                  '/content/sample_data/content/sample_data/aumento/val', (200, 200))

In [ ]:
model= func(MobileNet)

In [ ]:
ent_c(model, 5, ent_gen, val_gen, opt= 'Adam', perd= 'categorical_crossentropy', resumen= False)